In [2]:
import ifcopenshell
import ifcopenshell.util.element
import pandas as pd
from collections import Counter

In [3]:
ifc_file_doc = r'C:\Users\lucas.galicioli\ifc-classifier\notebooks\data\processed\PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00_modificado.ifc'

# 1. Carregar o arquivo IFC
ifc_file = ifcopenshell.open(ifc_file_doc)

print("Arquivo carregado com sucesso!")

# 2. Inspecionar os tipos de elementos e suas quantidades
products = ifc_file.by_type('IfcProduct')
element_counts = Counter(p.is_a() for p in products)

# 3. Imprimir Resumo
print("\nResumo dos Elementos no Modelo:")
for element_type, count in element_counts.items():
    print(f"- {element_type}: {count}")

Arquivo carregado com sucesso!

Resumo dos Elementos no Modelo:
- IfcValve: 488
- IfcPipeFitting: 4124
- IfcPipeSegment: 3192
- IfcDistributionPort: 15599
- IfcBuilding: 2
- IfcBuildingStorey: 17
- IfcSite: 2


In [4]:
# 1. Lista para armazenar os dados dos elementos
element_data = []

# 2. Iterar por todos os elementos físicos
products = ifc_file.by_type('IfcProduct')

for product in products:
    # 3. Extrair informações básicas
    element_info = {
        'ID':product.id(),
        'GlobalId':product.GlobalId,
        'Class':product.is_a(),
        'Name':getattr(product, 'Name', None)
    }
    # 4. Adicionar o dicionário à lista
    element_data.append(element_info)

# 5. Converter a lista em um DataFrame do Pandas
df = pd.DataFrame(element_data)

# Exibir as 5 primeiras linhas do nosso novo DataFrame
display(df.head())

,ID,GlobalId,Class,Name
0,3375,1n494Wk3P0mBiGCJTDJMTY,IfcValve,"Registro de Pressão:3/4"":2734253"
1,13441,1n494Wk3P0mBiGCJTDJHze,IfcValve,"Registro de Pressão:3/4"":2736295"
2,21237,1n494Wk3P0mBiGCJTDJHip,IfcValve,"Registro de Pressão:3/4"":2737404"
3,22775,1n494Wk3P0mBiGCJTDJHfw,IfcValve,"Registro de Pressão:3/4"":2737589"
4,24825,1n494Wk3P0mBiGCJTDJHU5,IfcValve,"Registro de Pressão:3/4"":2738250"


In [5]:
# Para ver as dimensões totais (linhas, colunas)
print(f"Dimensões do DataFrame: {df.shape}")

# Para ver apenas o número total de elementos (linhas)
print(f"Número total de elementos capturados: {len(df)}")

# Para gerar um resumo das contagens de cada classe, igual ao que fizemos antes, mas agora usando o DataFrame
print("\nResumo das classes de elementos no DataFrame:")
print(df['Class'].value_counts())

Dimensões do DataFrame: (23424, 4)
Número total de elementos capturados: 23424

Resumo das classes de elementos no DataFrame:
Class
IfcDistributionPort    15599
IfcPipeFitting          4124
IfcPipeSegment          3192
IfcValve                 488
IfcBuildingStorey         17
IfcBuilding                2
IfcSite                    2
Name: count, dtype: int64


In [6]:
element_data = []
for product in products:
    psets = ifcopenshell.util.element.get_psets(product)
    
    # Dicionário base com as informações que você já tem
    element_info = {
        'GlobalId': product.GlobalId,
        'Class': product.is_a(),
        'Name': getattr(product, 'Name', None) # Usar getattr para evitar erros se não houver nome
    }
    
    # Iterar sobre os Psets e adicionar cada propriedade ao dicionário
    for pset_name, properties in psets.items():
        for prop_name, prop_value in properties.items():
            # Cria um nome de coluna padronizado, ex: "Pset_PipeCommon.Reference"
            column_name = f"{pset_name}.{prop_name}"
            element_info[column_name] = prop_value
            
            
    element_data.append(element_info)

# Criar o DataFrame
# fillna(pd.NA) é bom para tratar células vazias de forma consistente
df = pd.DataFrame(element_data).fillna(pd.NA) 

display(df.head())

,GlobalId,Class,Name,Pset_EnvironmentalImpactIndicators.Reference,Pset_EnvironmentalImpactIndicators.id,Pset_ValveTypeCommon.Reference,Pset_ValveTypeCommon.id,PSET_RÔGGA.RÔGGA_DESCRIÇÃO,PSET_RÔGGA.RÔGGA_MACROLOG,PSET_RÔGGA.RÔGGA_EAP,...,Pset_BuildingElementProxyCommon.Reference,Pset_BuildingElementProxyCommon.IsExternal,Pset_BuildingElementProxyCommon.id,Pset_BuildingStoreyCommon.Reference,Pset_BuildingStoreyCommon.AboveGround,Pset_BuildingStoreyCommon.id,Pset_BuildingSystemCommon.Reference,Pset_BuildingSystemCommon.id,Pset_SiteCommon.Reference,Pset_SiteCommon.id
0,1n494Wk3P0mBiGCJTDJMTY,IfcValve,"Registro de Pressão:3/4"":2734253","Registro de Pressão:3/4""",3380.0,"Registro de Pressão:3/4""",3381.0,Registro de pressão,TOR1,TER,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN
1,1n494Wk3P0mBiGCJTDJHze,IfcValve,"Registro de Pressão:3/4"":2736295","Registro de Pressão:3/4""",13445.0,"Registro de Pressão:3/4""",13446.0,Registro de pressão,TOR1,TER,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN
2,1n494Wk3P0mBiGCJTDJHip,IfcValve,"Registro de Pressão:3/4"":2737404","Registro de Pressão:3/4""",21241.0,"Registro de Pressão:3/4""",21242.0,Registro de pressão,TOR1,TER,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN
3,1n494Wk3P0mBiGCJTDJHfw,IfcValve,"Registro de Pressão:3/4"":2737589","Registro de Pressão:3/4""",22779.0,"Registro de Pressão:3/4""",22780.0,Registro de pressão,TOR1,TER,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN
4,1n494Wk3P0mBiGCJTDJHU5,IfcValve,"Registro de Pressão:3/4"":2738250","Registro de Pressão:3/4""",24829.0,"Registro de Pressão:3/4""",24830.0,Registro de pressão,TOR1,TER,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN


In [7]:
# -*- coding: utf-8 -*-

import ifcopenshell
from ifcopenshell import api
from ifcopenshell.util import element
import os

# --- PASSO 1: CARREGAR O ARQUIVO IFC ---
ifc_file_path = r'C:\Users\lucas.galicioli\ifc-classifier\data\raw\PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00.ifc'#<-- AJUSTE O CAMINHO AQUI

if not os.path.exists(ifc_file_path):
    print(f"ERRO: O arquivo não foi encontrado em: {ifc_file_path}")
else:
    ifc_file = ifcopenshell.open(ifc_file_path)
    print(f"Arquivo '{os.path.basename(ifc_file_path)}' carregado com sucesso!")

    # --- PASSO 2: DEFINA O NOME DO PSET E AS PROPRIEDADES ---
    nome_do_pset = "RÖGGA_CLASSIFICAÇÃO"
    novas_propriedades = {
        "RÖGGA_CLS_SISTEMA": "HID-AGUA-FRIA",
        "RÖGGA_CLS_STATUS": "Verificado"
    }

    # --- PASSO 3: SELECIONE OS ELEMENTOS ALVO E APLIQUE O PSET ---
    elementos_alvo = ifc_file.by_type('IfcValve')
    
    if not elementos_alvo:
        print("\nAVISO: Nenhum elemento do tipo 'IfcValve' foi encontrado no modelo.")
    else:
        print(f"\nEncontrados {len(elementos_alvo)} elementos 'IfcValve'. Adicionando o Pset a cada um...")

        # Loop para aplicar o Pset a cada elemento
        for alvo in elementos_alvo:
            # ETAPA 3.1: Criar o Pset vazio e obter a referência para ele
            pset = ifcopenshell.api.run("pset.add_pset", 
                                        ifc_file, 
                                        product=alvo, 
                                        name=nome_do_pset)
            
            # ETAPA 3.2: Editar o Pset recém-criado para adicionar as propriedades
            ifcopenshell.api.run("pset.edit_pset", 
                                 ifc_file, 
                                 pset=pset, 
                                 properties=novas_propriedades)

        print(f"Pset '{nome_do_pset}' foi adicionado com sucesso a {len(elementos_alvo)} elementos.")

        # --- PASSO 4: VERIFIQUE O RESULTADO (em um elemento de amostra) ---
        print("\n--- Verificando o primeiro elemento modificado para confirmar a alteração: ---")
        primeiro_alvo_modificado = elementos_alvo[0]
        print(f"Elemento de amostra: {primeiro_alvo_modificado.Name} (ID: {primeiro_alvo_modificado.id()})")
        
        todos_os_psets_modificados = element.get_psets(primeiro_alvo_modificado, psets_only=True)

        for pset_name, properties in todos_os_psets_modificados.items():
            print(f"\n  -> Pset: '{pset_name}'")
            for prop_name, prop_value in properties.items():
                print(f"     - {prop_name}: {prop_value}")


        # --- PASSO 5: SALVE O ARQUIVO IFC MODIFICADO ---
        output_dir = 'data/processed'
        os.makedirs(output_dir, exist_ok=True)
        
        nome_arquivo_original = os.path.basename(ifc_file_path)
        novo_nome_arquivo = f"{os.path.splitext(nome_arquivo_original)[0]}_modificado.ifc"
        caminho_novo_arquivo = os.path.join(output_dir, novo_nome_arquivo)

        ifc_file.write(caminho_novo_arquivo)

        print(f"\nArquivo IFC modificado foi salvo em: {caminho_novo_arquivo}")
        

Arquivo 'PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00.ifc' carregado com sucesso!

Encontrados 488 elementos 'IfcValve'. Adicionando o Pset a cada um...
Pset 'RÖGGA_CLASSIFICAÇÃO' foi adicionado com sucesso a 488 elementos.

--- Verificando o primeiro elemento modificado para confirmar a alteração: ---
Elemento de amostra: Registro de Pressão:3/4":2734253 (ID: 3375)

  -> Pset: 'Pset_EnvironmentalImpactIndicators'
     - Reference: Registro de Pressão:3/4"
     - id: 3380

  -> Pset: 'Pset_ValveTypeCommon'
     - Reference: Registro de Pressão:3/4"
     - id: 3381

  -> Pset: 'PSET_RÔGGA'
     - RÔGGA_DESCRIÇÃO: Registro de pressão
     - RÔGGA_MACROLOG: TOR1
     - RÔGGA_EAP: TER
     - RÔGGA_SISTEMA: Água fria
     - RÔGGA_SEÇÃO: ø 25 x ø 25
     - id: 3387

  -> Pset: 'RÖGGA_CLASSIFICAÇÃO'
     - RÖGGA_CLS_SISTEMA: HID-AGUA-FRIA
     - RÖGGA_CLS_STATUS: Verificado
     - id: 324435

Arquivo IFC modificado foi salvo em: data/processed\PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00_